#### ISSUES:

* texts are repeating (solved)
* issues with some glosses (not lists)
* what to do about numeral / noun compounds? сом-зкьЫ-зтын

#### IDEAS: 

* think about 'io' indexes on nouns

In [1]:
import pympi
from bs4 import BeautifulSoup
import pandas as pd
from collections import Counter
import json
import re 
import pyconll
from os import walk
import itertools

### 1. Creating a conll-U file and adding the POS-tags

In [2]:
files = []
for (dirpath, dirnames, filenames) in walk('abaza_json'):
    files.extend(filenames)
    break

In [3]:
len(files)

25

Opening the json for individual text and parsing it.

In [4]:
tags = []
out = []
i = 1

for file in files:
    texts = []
    trans = []
    
    with open('abaza_json/' + file) as json_file:
        data = json.load(json_file)

    for element in data.get('sentences'): 
        sent = element.get('text')
        words = []

        for word in element.get('words'):
            glosses = word.get('ana')
            if isinstance(glosses, list):
                words.append((glosses[0].get('lex_acute'), glosses[0].get('gloss'), glosses[0].get('parts')))
        if len(words) > 0:
            texts.append([sent.replace('\n', ''), words])
        else:
            trans.append(sent.replace('\n', ''))

    for element, trans in zip(texts, trans):
        try:
            sent_id = i
            text_name = re.search(r'(.+).json', file).group(1)
            text = element[0]
            trans = trans

            string = '# sent_id = {}\n# text_name = {}\n# text = {}\n# ft_rus = {}\n'.format(sent_id, text_name, text, trans)
            out.append(string)

            k = 1
        
            for word in element[1]:
                gloss = re.sub(r'[а-я]|[А-Я]', '', word[1])
                gloss = re.sub(r'-', '|', gloss)
                gloss = re.sub(r'_', '', gloss)
                gloss = re.sub(r'\?', '', gloss)
                gloss = re.sub(r'\(', '', gloss)
                gloss = re.sub(r'\)', '', gloss)
                gloss = re.sub(r'ё', '', gloss)
                gloss = re.sub(r'\[', '', gloss)
                gloss = re.sub(r'\]', '', gloss)
                gloss = re.sub(r'(\|)+', '|', gloss) 
                gloss = re.sub(r'-$|^-', '', gloss)
                gloss = re.sub(r'\|$|^\|', '', gloss)

                pos = '_'

                if len(gloss)<2:
                    gloss = '_'
                    # experimental: try to assign to anything without gloss
                    # pos = 'NOUN'
                elif 'adv' in gloss:
                    pos = 'ADV'
                elif 'abs' in gloss:
                    pos = 'VERB'
                elif 'erg' in gloss:
                    pos = 'VERB'
                elif 'def' in gloss:
                    pos = 'NOUN'
                elif 'indf' in gloss: 
                    pos = 'NOUN'
                elif 'pl' in gloss: 
                    pos = 'NOUN'
                elif 'dcl' in gloss: 
                    pos = 'VERB'
                elif 'ins' in gloss: 
                    pos = 'NOUN'
                elif 'dist' in gloss: 
                    pos = 'PRON'
                elif 'msd' in gloss: 
                    pos = 'VERB'
                elif 'coord' in gloss: 
                    pos = 'CCONJ'
#                 elif ['abs', 'erg'] not in gloss and 'io' in gloss:
#                     pos = 'NOUN'
                else: 
                    pos = '_' 

                string = '{}\t{}\t_\t{}\t_\t{}\t_\t_\t_\tSegment={}|Gloss={}\n'.format(k, word[0], pos, gloss, word[2], word[1])
                out.append(string)
                k += 1

                tags.append(gloss)

            out.append('\n')

            i += 1
        
        except TypeError:
            print([i[1] for i in element[1]])

['Гумлокт', '3m.io-брат-indf', 'Кубиналокт', '3m.io-два-inc-ipf', '3m.io-брат', 'Инджичлокт-add', '3m.io-три-inc-ipf', '3m.io-брат', None]
['рЫцӏа', 'й-глАвн-у', None, None, None, None]
['плохой-adv', 'относиться', 'rel.erg-делать-ipf-pst.nfin rel.abs-3n.io-ben-плохой-pst.nfin', '1pl.io-государство', 'rel.abs-3n.io-ben-плохой-pst', '3sg.erg-говорить-dcl', 'десять-lnk-один-миллион-unit', 'доллар-ins', '3n.abs-csl-1sg.io-3m.erg-давать-ipf-pst', '3n.abs-neg.emp-3m.io-1sg.erg-neg-?-dcl', '3m.erg-говорить-dcl', None]
['1sg.abs-горец-cnc', 'мы', '1pl.io-возле', 'dist-похожий', 'neg.emp-быть-neg', '1sg.erg-сказал-dcl', 'dist.sg', 'rel.mnr-3m.io-dat-1sg.erg-сказать-pst.nfin', 'старуха-indf', 'ребёнок-add', '3f.io-спина', '3h.abs-loc-сидеть-lat-adv', '3h.abs-csl-идти-pst', 'хвать-quot', 'def-палка-шест', 'csl-3f.erg-взять-pst', 'def-filler', 'def.верёвка', '3h.abs-loc-перейти-pst', 'def-палка-add', 'loc-3f.erg-caus-падать-pst', '3h.abs-loc-loc.elat-войти-re-dcl', None]
['dist.pl', '1sg.erg-дела

Conll-u transformation:

In [5]:
data = ''.join(out)

In [6]:
print(data[1:1000])

 sent_id = 1
# text_name = 2018.07.16_BidzhevaTA_1932_retsept
# text = [tab1932_f] квтIАгъьгIважва гIатурЫчIвитIта
# ft_rus = [tab1932_f] Слив двадцать яиц
1	kʷṭáʁ’ʕʷaẑa	_	_	_	_	_	_	_	Segment=квтӏАгъьгӏважва|Gloss=яйцо-два-десять
2	ʕatwrə́ĉ̣əjṭta	_	ADV	_	csl|loc|2m.erg|caus|prs|dcl|adv	_	_	_	Segment=гӏатурЫчӏвитӏта|Gloss=csl-loc-2m.erg-caus-литься-prs-dcl-adv

# sent_id = 2
# text_name = 2018.07.16_BidzhevaTA_1932_retsept
# text = ауИ аъарА столОвая лОжка дзЫта йАлаучвахитI
# ft_rus = Туда вливаешь столько же столовых ложек воды.
1	awə́j	_	PRON	_	dist.sg	_	_	_	Segment=ауи|Gloss=dist.sg
2	aʔará	_	_	_	_	_	_	_	Segment=аъарА|Gloss=столько
3	столовая	_	_	_	_	_	_	_	Segment=столовая|Gloss=столовая
4	ложка	_	_	_	_	_	_	_	Segment=ложка|Gloss=ложка
5	ʒə́ta	_	ADV	_	adv	_	_	_	Segment=дзЫ-та|Gloss=вода-adv
6	jálawĉaχəjṭ	_	VERB	_	3n.abs|3n.io|loc|2m.erg|re|prs|dcl	_	_	_	Segment=й-А-ла-у-чва-х-и-тӏ|Gloss=3n.abs-3n.io-loc-2m.erg-вливать-re-prs-dcl

# sent_id = 3
# text_name = 2018.07.16_Bidzh


In [7]:
with open("corpus.conllu", "w") as text_file:
    text_file.write(data)

### 2. More detailled annotation attempts

Working with tags unique tags:

In [8]:
len(set(tags))

1283

In [9]:
tags_clean = []
for elemnt in tags:
    tags_clean.append(elemnt.split('|'))

merged = list(itertools.chain.from_iterable(tags_clean))
len(set(merged))

224

### 3. Adding propper tags 

In [7]:
!cat corpus_annotation.conllu | python3 conllu-feats.py abaza_proper.udx > example_out.conllu

5.0 {'def'} {'Definite=Def', 'NOUN'}
5.0 {'indef'} {'Definite=Ind', 'NOUN'}
5.0 {'ins'} {'Case=Ins'}
5.0 {'dat'} {'Case=Dat'}
5.0 {'neg'} {'Polarity=Neg'}
5.0 {'pl'} {'Number=Plur', 'NOUN'}
5.0 {'pl.h'} {'Number=Plur', 'NOUN'}
5.0 {'unit'} {'Number=Count'}
5.0 {'cln'} {'NUM'}
5.0 {'clh'} {'NUM'}
5.0 {'clh'} {'NUM'}
5.0 {'adv'} {'ADV'}
5.0 {'1sg.erg'} {'Person[erg]=1', 'Number[erg]=Sing'}
5.0 {'1sg.abs'} {'Number[abs]=Sing', 'Person[abs]=1'}
5.0 {'1sg.io'} {'Number[dat]=Sing', 'Person[dat]=1'}
5.0 {'2m.erg'} {'Gender[erg]=Masc', 'Person[erg]=2', 'Number[erg]=Sing'}
5.0 {'2m.abs'} {'Number[abs]=Sing', 'Person[abs]=2', 'Gender[abs]=Masc'}
5.0 {'2m.io'} {'Person[dat]=2', 'Number[dat]=Sing', 'Gender[dat]=Masc'}
5.0 {'2f.erg'} {'Gender[erg]=Fem', 'Person[erg]=2', 'Number[erg]=Sing'}
5.0 {'2f.abs'} {'Number[abs]=Sing', 'Gender[abs]=Fem', 'Person[abs]=2'}
5.0 {'2f.io'} {'Gender[dat]=Fem', 'Person[dat]=2', 'Number[dat]=Sing'}
5.0 {'2n.erg'} {'Gender[erg]=Neut', 'Person[erg]=2', 'Number[erg]=Sin

> {'adv', '_', '3f.erg'}
- {'adv', '_', '3f.erg'} {'adv'} {'_', '3f.erg'} {'ADV'} ||| _  _
- {'_', '3f.erg'} {'3f.erg'} {'_'} {'Gender[erg]=Fem', 'Person[erg]=3', 'Number[erg]=Sing'} ||| ADV  _
> {'def', '_'}
- {'def', '_'} {'def'} {'_'} {'Definite=Def', 'NOUN'} ||| _  _
> {'3n.io', '_'}
- {'3n.io', '_'} {'3n.io'} {'_'} {'Person[dat]=3', 'Number[dat]=Sing', 'Gender[dat]=Neut'} ||| _  _
> {'fut', '2pl.erg', 'dcl', '3h.abs', '_', 'ipf'}
- {'fut', '2pl.erg', 'dcl', '3h.abs', '_', 'ipf'} {'2pl.erg'} {'fut', 'dcl', '3h.abs', '_', 'ipf'} {'Number[erg]=Plur', 'Person[erg]=2'} ||| _  _
- {'fut', 'dcl', '3h.abs', '_', 'ipf'} {'ipf'} {'fut', '3h.abs', '_', 'dcl'} {'VERB', 'Tense=Imp'} ||| _ Number[erg]=Plur|Person[erg]=2 _
- {'fut', '3h.abs', '_', 'dcl'} {'dcl'} {'fut', '3h.abs', '_'} {'Tense=Pst', 'VERB'} ||| VERB Number[erg]=Plur|Person[erg]=2|Tense=Imp _
- {'fut', '3h.abs', '_'} {'fut'} {'3h.abs', '_'} {'Tense=Fut'} ||| VERB Number[erg]=Plur|Person[erg]=2|Tense=Imp|Tense=Pst _
> {'med.loc', '

> {'adv', '_', 'msd'}
- {'adv', '_', 'msd'} {'adv'} {'_', 'msd'} {'ADV'} ||| _  _
- {'_', 'msd'} {'msd'} {'_'} {'VerbForm=Vnoun', 'VERB'} ||| ADV  _
> {'rel.abs', '1sg.io', '_', 'prs.nfin'}
- {'rel.abs', '1sg.io', '_', 'prs.nfin'} {'1sg.io'} {'rel.abs', '_', 'prs.nfin'} {'Number[dat]=Sing', 'Person[dat]=1'} ||| _  _
> {'adv', '_'}
- {'adv', '_'} {'adv'} {'_'} {'ADV'} ||| _  _
> {'2m.abs', '1sg.erg', '_', 'npst.dcl'}
- {'2m.abs', '1sg.erg', '_', 'npst.dcl'} {'1sg.erg'} {'2m.abs', 'npst.dcl', '_'} {'Person[erg]=1', 'Number[erg]=Sing'} ||| _  _
- {'2m.abs', 'npst.dcl', '_'} {'2m.abs'} {'npst.dcl', '_'} {'Number[abs]=Sing', 'Person[abs]=2', 'Gender[abs]=Masc'} ||| _ Person[erg]=1|Number[erg]=Sing _
- {'npst.dcl', '_'} {'npst.dcl'} {'_'} {'Tense=Pres'} ||| _ Person[erg]=1|Number[erg]=Sing|Number[abs]=Sing|Person[abs]=2|Gender[abs]=Masc _
> {'3m.erg', '_', 'pst'}
- {'3m.erg', '_', 'pst'} {'3m.erg'} {'_', 'pst'} {'Person[erg]=3', 'Gender[erg]=Masc', 'Number[erg]=Sing'} ||| _  _
- {'_', 'pst'}

> {'_', '3n.abs', 'csl'}
- {'_', '3n.abs', 'csl'} {'3n.abs'} {'_', 'csl'} {'Person[abs]=3', 'Number[abs]=Sing', 'Gender[abs]=Neut'} ||| _  _
> {'.', 'neg', 'cond.real', '3n.abs', '_'}
- {'.', 'neg', 'cond.real', '3n.abs', '_'} {'neg'} {'cond.real', '_', '.', '3n.abs'} {'Polarity=Neg'} ||| _  _
- {'cond.real', '_', '.', '3n.abs'} {'3n.abs'} {'_', '.', 'cond.real'} {'Person[abs]=3', 'Number[abs]=Sing', 'Gender[abs]=Neut'} ||| _ Polarity=Neg _
- {'_', '.', 'cond.real'} {'cond.real'} {'.', '_'} {'Mood=Cnd'} ||| _ Polarity=Neg|Person[abs]=3|Number[abs]=Sing|Gender[abs]=Neut _
> {'plh', '_', 'pl', 'add'}
- {'plh', '_', 'pl', 'add'} {'pl'} {'plh', '_', 'add'} {'Number=Plur', 'NOUN'} ||| _  _
> {'neg.emp', 'pst.nfin', 'dcl', 'neg', '_'}
- {'neg.emp', 'pst.nfin', 'dcl', 'neg', '_'} {'neg'} {'neg.emp', '_', 'pst.nfin', 'dcl'} {'Polarity=Neg'} ||| _  _
- {'neg.emp', '_', 'pst.nfin', 'dcl'} {'dcl'} {'neg.emp', '_', 'pst.nfin'} {'Tense=Pst', 'VERB'} ||| _ Polarity=Neg _
> {'_', 'dist.sg'}
> {'_'}
>

> {'caus', '3h.abs', 'imp', 'adv', '_', 'csl'}
- {'caus', '3h.abs', 'imp', 'adv', '_', 'csl'} {'adv'} {'caus', '3h.abs', 'imp', '_', 'csl'} {'ADV'} ||| _  _
- {'caus', '3h.abs', 'imp', '_', 'csl'} {'caus'} {'3h.abs', '_', 'imp', 'csl'} {'VERB', 'Voice=Cau'} ||| ADV  _
> {'_'}
> {'_', 'imp', '3n.abs'}
- {'_', 'imp', '3n.abs'} {'3n.abs'} {'_', 'imp'} {'Person[abs]=3', 'Number[abs]=Sing', 'Gender[abs]=Neut'} ||| _  _
> {'_', '3n.abs', 'dcl'}
- {'_', '3n.abs', 'dcl'} {'3n.abs'} {'_', 'dcl'} {'Person[abs]=3', 'Number[abs]=Sing', 'Gender[abs]=Neut'} ||| _  _
- {'_', 'dcl'} {'dcl'} {'_'} {'Tense=Pst', 'VERB'} ||| _ Person[abs]=3|Number[abs]=Sing|Gender[abs]=Neut _
> {'1sg.erg', '_', 'dcl'}
- {'1sg.erg', '_', 'dcl'} {'1sg.erg'} {'_', 'dcl'} {'Person[erg]=1', 'Number[erg]=Sing'} ||| _  _
- {'_', 'dcl'} {'dcl'} {'_'} {'Tense=Pst', 'VERB'} ||| _ Person[erg]=1|Number[erg]=Sing _
> {'_', '3n.abs', 'dcl'}
- {'_', '3n.abs', 'dcl'} {'3n.abs'} {'_', 'dcl'} {'Person[abs]=3', 'Number[abs]=Sing', 'Gender[